In [1]:
%tensorflow_version 2.x

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd


In [ ]:
#Dataset- IRIS dataset
#3 different classes of species -Setosa, Versicolor, Virginica
#information about each flower is given-
#sepal length, sepal width, petal length, petal width

In [3]:
COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [4]:
#Load the dataset from keras
#keras is a module inside tensorflow
train_path = tf.keras.utils.get_file("iris_training.csv", 
  "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", 
  "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

#Read the dataset
train = pd.read_csv(train_path, names=COLUMN_NAMES, header=0) #header=0 ->row 0 is column name/header
test = pd.read_csv(test_path, names=COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
#pop the Species column off -Species is label

train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() #species column is gone

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [7]:
train.shape #120 entries with 4 features

(120, 4)

In [8]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [9]:
test_y.head()

0    1
1    2
2    0
3    1
4    1
Name: Species, dtype: int64

In [10]:
#Input Function
def input_fn(features, labels, training=True, batch_size=256):
  #convert inputs to dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #shuffle and repeat if you are in training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

In [11]:
#Feature Columns - they describe how to use the input
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
#Build the model
#for classification we have different estimators/models
#DNNClassifier (Deep Neural Network), LinearClassifier
#DNNClassifier-best choice->we may not be able to find a linear correspondence in our data 


In [12]:
#Build DNN model with 2 hidden layers with 30 and 10 hidden nodes each
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    #2 hidden layers pf 30 and 10 nodes resp
    hidden_units=[30,10],
    #the model must choose between 3 classes
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9z48hj4g', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
#Train the model
#lambda used to avoid creating an inner input function
#lambda-anonymous fn
#example of lambda ->
#x = lambda: print("hii")
#x()  valid
classifier.train(
    input_fn = lambda: input_fn(train, train_y, training=True),
    steps=5000  #similar to epoch- it will go through dataset until 5000 things have been looked at
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9z48hj4g/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmp9z48hj4g/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.7030529, step = 5000
INFO:tensorflow:global_step/sec: 474.65
INFO:tensorflow:loss = 0.7045547, step = 5100 (0.212 sec)
INFO:tensorflow:global_step/sec: 555.804
INFO:tensorflow:loss = 0.69001865, step = 5200 (0.183 sec)
INFO:tensorflow:global_step/sec: 522.076
INFO:tensorflow:loss = 0.68547535, step = 5300 (0.192 sec)
INFO:tensorflow:global_step/sec: 578.348


In [ ]:
#loss - 0.46 ->the lower the loss the better

In [15]:
#Evaluation of model
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False)
    )

#Accuracy
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-07-02T13:12:00
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9z48hj4g/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.26208s
INFO:tensorflow:Finished evaluation at 2021-07-02-13:12:00
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.8666667, average_loss = 0.5456036, global_step = 10000, loss = 0.5456036
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmp9z48hj4g/model.ckpt-10000

Test set accuracy: 0.867



In [ ]:
#Accuracy-86.7%

In [24]:
#Predictions

def input_fn(features, batch_size=256):
  #convert inputs to a dataset without labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

f = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please enter numeric values of the flower:")
for i in f:
  valid = True
  while valid:
    val = input(i + ": ")
    if not val.isdigit():
      valid = False
    
    predict[i] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('\nPrediction is "{}" ({:.1f}%)'.format(
      SPECIES[class_id], 100*probability
  ))

Please enter numeric values of the flower:
SepalLength: 2.4
SepalWidth: 2.6
PetalLength: 6.5
PetalWidth: 6.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9z48hj4g/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([0.6417767, 1.04211  , 1.6966567], dtype=float32), 'probabilities': array([0.18642968, 0.2782131 , 0.5353572 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}

Prediction is "Virginica" (53.5%)
